In [36]:
from __future__ import annotations
from dataclasses import dataclass, field, fields
import functools


def rsetattr(obj, attr, val):
    pre, _, post = attr.rpartition('.')
    return setattr(rgetattr(obj, pre) if pre else obj, post, val)

def rgetattr(obj, attr, *args):
    def _getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(_getattr, [obj] + attr.split('.'))


@dataclass
class BgSub:
    sigma: float = field(default_factory=float)
    size: int = field(default_factory=int)
    overwrite: bool = field(default_factory=bool)
    
@dataclass
class ChanShift:
    reg_channel: str = field(default_factory=str)
    reg_mtd: str = field(default_factory=str)
    overwrite: bool = field(default_factory=bool)
    
@dataclass
class Register:
    reg_channel: str = field(default_factory=str)
    reg_mtd: str = field(default_factory=str)
    reg_ref: str = field(default_factory=str)
    overwrite: bool = field(default_factory=bool)
    
@dataclass
class Blur:
    kernel: tuple[int] = field(default_factory=tuple)
    sigma: int = field(default_factory=int)
    img_fold_src: str = field(default_factory=str)
    overwrite: bool = field(default_factory=bool)

def unpack_settings(input_settings: dict)-> Settings:
    settings = Settings()
    if input_settings['run_bg_sub']:
        settings.bg_sub = BgSub(**input_settings['bg_sub'])
    if input_settings['run_chan_shift']:
        settings.chan_shift = ChanShift(**input_settings['chan_shift'])
    if input_settings['run_register']:
        settings.register = Register(**input_settings['register'])
    if input_settings['run_blur']:
        settings.blur = Blur(**input_settings['blur'])
    return settings


@dataclass
class Settings:
    settings: dict
    bg_sub: dict = field(init=False)
    chan_shift: dict = field(init=False)
    register: dict = field(init=False)
    blur: dict = field(init=False)
    
    def __post_init__(self)-> None:
        if self.settings['run_bg_sub']:
            self.bg_sub = self.settings['bg_sub']
        if self.settings['run_chan_shift']:
            self.chan_shift = self.settings['chan_shift']
        if self.settings['run_register']:
            self.register = self.settings['register']
        if self.settings['run_blur']:
            self.blur = self.settings['blur']
        self.update_overwrite()
        
    def update_overwrite(self)-> None:
        active_branches = [f.name for f in fields(self) if hasattr(self,f.name) and f.name != 'settings']
        current_overwrite = [getattr(self,f)['overwrite'] for f in active_branches]

        # Get the new overwrite list, if the previous is true then change the next to true, else keep the same
        new_overwrite = []; is_False = True
        for i in range(len(current_overwrite)):
            if current_overwrite[i] == False and is_False:
                new_overwrite.append(current_overwrite[i])
            elif current_overwrite[i] == True and is_False:
                new_overwrite.append(current_overwrite[i])
                is_False = False
            elif not is_False:
                new_overwrite.append(True)# Update the overwrite attribute
        
        # Update the overwrite attribute
        for i,branch in enumerate(active_branches):
            temp_dict = getattr(self,branch)
            temp_dict['overwrite'] = new_overwrite[i]
            setattr(self,branch,temp_dict)


settings = settings = {
    "run_bg_sub": True,
    "run_chan_shift": True,
    "run_register": True,
    "run_blur": True,
    
    "bg_sub": {"sigma": 0.0,"size": 7,"overwrite": True},
    "chan_shift": {"reg_channel": "C1","reg_mtd": "rigid","overwrite": False},
    "register": {"reg_channel": "C1","reg_mtd": "rigid","reg_ref": "C1","overwrite": False},
    "blur": {"kernel": (5,5),"sigma": 0,"img_fold_src": "","overwrite": True}}

s = Settings(settings)
print(f"{s.chan_shift['overwrite'] = }")
s.update_overwrite()
print(f"{s.chan_shift['overwrite'] = }")

s.chan_shift['overwrite'] = False
s.chan_shift['overwrite'] = True


In [8]:
def myfunc(a,b):
    return a+b, a-b

def myfunc2(a,b,c):
    return *myfunc(a,b), c

print(myfunc2(1,2,3))

(3, -1, 3)


In [1]:
d = {'a':1,'b':2}
d['a'] = d['b'] = 3
print(d)

{'a': 3, 'b': 3}


In [16]:
s = "GFP_s01_f0001_z0001.tif"

for i in range(25):
    temp = s.split('_z')
    file_type = temp[1].split('.')[-1]
    new_z = f"_z{i+1:04d}.{file_type}"
    new_file = temp[0] + new_z
    print(new_file)

GFP_s01_f0001_z0001.tif
GFP_s01_f0001_z0002.tif
GFP_s01_f0001_z0003.tif
GFP_s01_f0001_z0004.tif
GFP_s01_f0001_z0005.tif
GFP_s01_f0001_z0006.tif
GFP_s01_f0001_z0007.tif
GFP_s01_f0001_z0008.tif
GFP_s01_f0001_z0009.tif
GFP_s01_f0001_z0010.tif
GFP_s01_f0001_z0011.tif
GFP_s01_f0001_z0012.tif
GFP_s01_f0001_z0013.tif
GFP_s01_f0001_z0014.tif
GFP_s01_f0001_z0015.tif
GFP_s01_f0001_z0016.tif
GFP_s01_f0001_z0017.tif
GFP_s01_f0001_z0018.tif
GFP_s01_f0001_z0019.tif
GFP_s01_f0001_z0020.tif
GFP_s01_f0001_z0021.tif
GFP_s01_f0001_z0022.tif
GFP_s01_f0001_z0023.tif
GFP_s01_f0001_z0024.tif
GFP_s01_f0001_z0025.tif


In [4]:

from os import PathLike
from tifffile import TiffFile, imread, imwrite


def get_tif_meta(img_path: PathLike) -> dict:
    # Open tif and read meta
    with TiffFile(img_path) as tif:
        imagej_meta = tif.imagej_metadata
        imagej_meta['axes'] = tif.series[0].axes
        for page in tif.pages: # Add additional meta
            for tag in page.tags:
                if tag.name in ['ImageWidth','ImageLength',]:
                    imagej_meta[tag.name] = tag.value
                if tag.name in ['XResolution','YResolution']:
                    imagej_meta[tag.name] = tag.value[0]/tag.value[1]
    
    # if 'frames' not in imagej_meta: imagej_meta['frames'] = 1
    
    # if 'channels' not in imagej_meta: imagej_meta['channels'] = 1
    
    # if 'slices' not in imagej_meta: imagej_meta['slices'] = 1
    
    # if 'finterval' not in imagej_meta: imagej_meta['finterval'] = 0
    
    # imagej_meta['n_series'] = 1
    return imagej_meta

img_path = '/home/Test_images/tiff/Run4/c4z1t91v1.tif'
save_test = '/home/Test_images/tiff/Run4/test.tif'
meta = get_tif_meta(img_path)
img = imread(img_path)
imwrite(save_test,img[0,...],metadata={'channels':4, 'original_frames':91,'frames':1,'finterval': 10,'unit':'um'},resolution=(3.076923,2),imagej=True)
with TiffFile(save_test) as tif:
    meta_test = tif.imagej_metadata
    for page in tif.pages:
        for tag in page.tags:
            if tag.name=='ImageDescription':
                print(tag.value)

ImageJ=1.11a
images=4
channels=4
hyperstack=true
mode=grayscale
original_frames=91
finterval=10
unit=um


In [2]:
meta_test

{'ImageJ': '1.11a',
 'images': 4,
 'channels': 4,
 'hyperstack': True,
 'mode': 'grayscale',
 'original_frames': 91,
 'finterval': 10}